In [2]:
%reload_ext autoreload
%autoreload 2

# Use aligner_v7 kernel

import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    "/fsx_0/shared/conda/aligner_20241030/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
device = "cuda:0"

In [3]:
import torch

# inference/server
from llm_mm_aligner.lib.encoders.metaclip_native_resolution_inference import MetaCLIP_NAVIT as MetaCLIP_NAVIT_INF

# training + readout
from llm_mm_aligner.lib.encoders.metaclip_navit import MetaCLIP_NAVIT
from PIL import Image
from torch import nn
from torchvision import transforms

/opt/hpcaas/.mounts/fs-036153e63d56f4dc2/home/tranx/conda/envs/aligner_v7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Init model

In [4]:
MODEL_CHECKPOINT = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
TEST_IMAGE = "/fsx_0/dataset01/coco/val2014/COCO_val2014_000000485307.jpg"

# ViT-H
metaclip_config = {
    "ckpt_path": MODEL_CHECKPOINT,
    "image_size": 336,  # image_size can also be 448, since the position embedding is rope_2d
    "patch_size": 14,
    "width": 1408,
    "mlp_ratio": 4.0,
    "layers": 34,
    "heads": 16,
    "load_ckpt": True,
    "relative_pos_embed_type": "rope_2d",
}


CLIP_MEAN = (0.4814546, 0.4578275, 0.40821073)
CLIP_STD = (0.2686295, 0.2613025, 0.2757711)

preprocess = transforms.Compose(
    [
        transforms.Resize(metaclip_config["image_size"]),
        transforms.CenterCrop(metaclip_config["image_size"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4814546, 0.4578275, 0.40821073),
            std=(0.2686295, 0.2613025, 0.2757711),
        ),
    ]
)

In [5]:
image = Image.open(TEST_IMAGE).convert("RGB")
preprocessed_image = preprocess(image).unsqueeze(0).to(device)

In [6]:
model1 = MetaCLIP_NAVIT(**metaclip_config).to(device)
model1.eval()

MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT Visual Pretrained Model Missing keys: []
MetaCLIP_NAVIT Visual Pretrained Model Unexpected keys: []


MetaCLIP_NAVIT(
  (conv1): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-33): 34 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionXformer4x(
          (out_proj): Linear(in_features=1408, out_features=1408, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1408, out_features=5632, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=5632, out_features=1408, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
)

In [7]:
with torch.no_grad():
    output1 = model1(preprocessed_image)

output1.shape

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_navit.py:337: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


torch.Size([1, 576, 1408])

In [8]:
model2 = MetaCLIP_NAVIT_INF(**metaclip_config).to(device)
model2.eval()

Unexpected keys: []


MetaCLIP_NAVIT(
  (conv1): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-33): 34 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionXformer4x(
          (out_proj): Linear(in_features=1408, out_features=1408, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1408, out_features=5632, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=5632, out_features=1408, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
)

In [9]:
with torch.no_grad():
    output2 = model2(preprocessed_image)

output2.shape

/fsx_0/user/tranx/rsync/llm_mm_aligner/lib/encoders/metaclip_native_resolution_inference.py:899: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(


torch.Size([1, 576, 1408])

In [10]:
output1

tensor([[[ 2.4852e+00,  8.1081e-01,  6.0595e-01,  ..., -1.9258e+00,
          -1.2234e-01,  4.2518e+00],
         [ 3.5210e-01,  3.2244e+00, -6.3592e+00,  ..., -3.6949e-01,
          -1.1734e+00,  3.9925e+00],
         [ 2.7544e+00,  7.4645e-01, -4.8141e+00,  ..., -1.6364e+00,
          -5.5741e+00,  6.2823e-03],
         ...,
         [-1.8769e+00,  2.5796e+00,  2.5895e+00,  ...,  5.2259e+00,
          -6.4294e-01,  4.7235e+00],
         [ 7.9809e-01,  1.3053e+00, -1.4412e+00,  ..., -1.3024e+00,
          -6.2229e+00, -9.8805e-01],
         [-2.4709e+00, -3.5142e-01,  5.6059e-02,  ..., -3.4691e+00,
          -3.0556e+00,  3.3854e+00]]], device='cuda:0')

In [11]:
output2

tensor([[[ 2.4852e+00,  8.1081e-01,  6.0595e-01,  ..., -1.9258e+00,
          -1.2234e-01,  4.2518e+00],
         [ 3.5210e-01,  3.2244e+00, -6.3592e+00,  ..., -3.6949e-01,
          -1.1734e+00,  3.9925e+00],
         [ 2.7544e+00,  7.4645e-01, -4.8141e+00,  ..., -1.6364e+00,
          -5.5741e+00,  6.2823e-03],
         ...,
         [-1.8769e+00,  2.5796e+00,  2.5895e+00,  ...,  5.2259e+00,
          -6.4294e-01,  4.7235e+00],
         [ 7.9809e-01,  1.3053e+00, -1.4412e+00,  ..., -1.3024e+00,
          -6.2229e+00, -9.8805e-01],
         [-2.4709e+00, -3.5142e-01,  5.6059e-02,  ..., -3.4691e+00,
          -3.0556e+00,  3.3854e+00]]], device='cuda:0')

In [12]:
diff = torch.abs(output1 - output2)
# Calculate the norm of the difference
norm_diff = torch.norm(diff)
print(f"Norm of the difference: {norm_diff}")

Norm of the difference: 0.0
